In [3]:
!pip install pandas numpy scikit-learn

  Using cached pandas-2.0.1-cp39-cp39-macosx_11_0_arm64.whl (10.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 6.9 MB/s eta 0:00:0000:0100:01
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [5]:
epl = pd.read_csv("EPL_20_21.csv")
print(len(epl))
#epl.describe()
epl.head()

532


,Name,Club,Nationality,Position,Age,Matches,Starts,Mins,Goals,Assists,Passes_Attempted,Perc_Passes_Completed,Penalty_Goals,Penalty_Attempted,xG,xA,Yellow_Cards,Red_Cards
0,Mason Mount,Chelsea,ENG,"MF,FW",21,36,32,2890,6,5,1881,82.3,1,1,0.21,0.24,2,0
1,Edouard Mendy,Chelsea,SEN,GK,28,31,31,2745,0,0,1007,84.6,0,0,0.00,0.00,2,0
2,Timo Werner,Chelsea,GER,FW,24,35,29,2602,6,8,826,77.2,0,0,0.41,0.21,2,0
3,Ben Chilwell,Chelsea,ENG,DF,23,27,27,2286,3,5,1806,78.6,0,0,0.10,0.11,3,0
4,Reece James,Chelsea,ENG,DF,20,32,25,2373,1,2,1987,85.0,0,0,0.06,0.12,3,0


In [6]:
teams = list(epl["Club"].unique())

matches = []

for i in range(len(teams)):
    for j in range(len(teams)):
        if i != j:
            matches.append([teams[i], teams[j]])

matches = pd.DataFrame(matches, columns=["Home", "Away"])

#epl[epl['Club'] == 'Manchester United']
print(len(matches))

def get_matches(team):
    return np.concatenate([
        np.array(matches[matches["Home"] == team].index),
        np.array(matches[matches["Away"] == team].index)])

team_matches = {t : get_matches(t) for t in teams}


380


In [8]:

def recompute_match_players():

    match_player = np.zeros((len(matches), 2 * len(epl) + 2 + 1 + 3)) # +2 = home, away, +1 won, + 3 odds home draw away

    for i, player in epl.iterrows():
        player_matches = np.random.choice(team_matches[player['Club']], player['Matches'], replace=False)
        match_player[player_matches, i] = 1

        goals = np.random.choice(player_matches, player['Goals'], replace=True)

        for k in goals:
            match_player[k, len(epl) + i] += 1 

    return match_player

match_player = recompute_match_players()

def get_match_id(home, away):
        return np.array(matches[(matches["Home"] == home) & (matches["Away"] == away)].index)[0]

def get_score(home, away):
    mid = get_match_id(home, away)
    home_players = np.array(epl[epl["Club"] == home].index)
    away_players = np.array(epl[epl["Club"] == away].index)

    return (mid, np.sum(match_player[mid, len(epl) + home_players]), np.sum(match_player[mid, len(epl) + away_players]))


print(get_score("Chelsea", "Manchester United"))
print(get_score("Manchester United", "Chelsea"))


(1, 1.0, 0.0)
(38, 6.0, 4.0)


In [9]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=(len(epl), )),
        #tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                26650     
                                                                 
 dense_1 (Dense)             (None, 3)                 153       
                                                                 
Total params: 26,803
Trainable params: 26,803
Non-trainable params: 0
_________________________________________________________________


In [10]:
for epoch in range(0, 200):
    print("Epoch", epoch)

    match_player = recompute_match_players()

    for i, r in matches.iterrows():

        ht = r['Home']
        at = r['Away']

        mid, score_home, score_away = get_score(ht, at)

        # lexicographic order
        # always show in the same order the Y column
        if (ht < at):
            first_score = score_home
            second_score = score_away
        else:
            second_score = score_home
            first_score = score_away

        match_player[mid, 2 * len(epl)] = first_score
        match_player[mid, 2 * len(epl) + 1] = second_score
        match_player[mid, 2 * len(epl) + 2] = int(first_score > second_score)
        match_player[mid, 2 * len(epl) + 3] = int(first_score == second_score)
        match_player[mid, 2 * len(epl) + 4] = int(first_score < second_score)

    idxs = list(range(0, len(epl))) # players
    start_label = 2 * len(epl) + 2
    idxs += [start_label, start_label + 1, start_label + 2] # score

    dataset = tf.data.Dataset.from_tensor_slices(match_player[:, idxs]).map(lambda x: (x[:-3], x[-3:])).shuffle(1024).batch(64)

    #for k, v in dataset.take(1):
    #    print(k, v)

    model.fit(dataset, epochs=1, verbose=2)



Epoch 0
6/6 - 0s - loss: 1.1032 - accuracy: 0.3711 - 172ms/epoch - 29ms/step


2023-05-01 15:18:55.566074: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1
6/6 - 0s - loss: 1.0898 - accuracy: 0.4053 - 15ms/epoch - 2ms/step
Epoch 2
6/6 - 0s - loss: 1.0978 - accuracy: 0.3868 - 18ms/epoch - 3ms/step
Epoch 3
6/6 - 0s - loss: 1.0761 - accuracy: 0.4158 - 15ms/epoch - 3ms/step
Epoch 4
6/6 - 0s - loss: 1.0814 - accuracy: 0.4079 - 16ms/epoch - 3ms/step
Epoch 5
6/6 - 0s - loss: 1.0661 - accuracy: 0.4237 - 15ms/epoch - 2ms/step
Epoch 6
6/6 - 0s - loss: 1.0781 - accuracy: 0.4289 - 16ms/epoch - 3ms/step
Epoch 7
6/6 - 0s - loss: 1.0728 - accuracy: 0.4316 - 16ms/epoch - 3ms/step
Epoch 8
6/6 - 0s - loss: 1.0836 - accuracy: 0.4237 - 15ms/epoch - 3ms/step
Epoch 9
6/6 - 0s - loss: 1.0809 - accuracy: 0.4342 - 16ms/epoch - 3ms/step
Epoch 10
6/6 - 0s - loss: 1.0538 - accuracy: 0.4289 - 16ms/epoch - 3ms/step
Epoch 11
6/6 - 0s - loss: 1.0690 - accuracy: 0.4421 - 15ms/epoch - 2ms/step
Epoch 12
6/6 - 0s - loss: 1.0618 - accuracy: 0.4500 - 15ms/epoch - 2ms/step
Epoch 13
6/6 - 0s - loss: 1.0578 - accuracy: 0.4342 - 14ms/epoch - 2ms/step
Epoch 14
6/6 - 0s - l

In [11]:
v = np.empty([1, len(epl)])
match_player = recompute_match_players()


def predict(h, a):
    mid = get_match_id(h, a)
    v[0] = match_player[mid, :len(epl)]
    p = model.predict(v)

    mid, home, away = get_score(h, a)

    if (h < a):
        print(h, a, home, away, 1/p)
    else:
        print(a, h, away, home, 1/p)


predict("Manchester City", "Chelsea")
predict("Chelsea", "Manchester City")
predict("Manchester City", "Southampton")
predict("Southampton", "Manchester City")
predict("Southampton", "Chelsea")
predict("Chelsea", "Southampton")

1/1 [==============================] - 0s 34ms/step
Chelsea Manchester City 2.0 2.0 [[3.6205077 4.951697  1.916279 ]]
1/1 [==============================] - 0s 10ms/step
Chelsea Manchester City 1.0 2.0 [[4.4548073 5.002399  1.7372594]]
1/1 [==============================] - 0s 10ms/step
Manchester City Southampton 1.0 3.0 [[1.7484107 5.4051485 4.114495 ]]
1/1 [==============================] - 0s 10ms/step
Manchester City Southampton 0.0 2.0 [[1.8436499 5.20579   3.7664237]]
1/1 [==============================] - 0s 9ms/step
Chelsea Southampton 1.0 0.0 [[2.054378  3.8595972 3.9348338]]
1/1 [==============================] - 0s 9ms/step
Chelsea Southampton 1.0 0.0 [[2.0968943 3.8357725 3.8109665]]
